In [10]:
!pip install timm

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from tqdm.notebook import tqdm
import timm
import torch.optim.lr_scheduler as lr_scheduler
import math
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import pandas as pd
import numpy as np
import pickle
# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
train_transform = transforms.Compose([
    # Geometric transformations
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),  # Flip the image with probability=0.5
    transforms.RandomVerticalFlip(p=0.5),  # Flip image vertically with probability=0.5
    transforms.RandomRotation(30),  # Rotate the image up to 30 degrees
    transforms.RandomRotation(60),  # Rotate the image up to 90 degrees
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),  # Crop and resize

    # Color transformations
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),

    # Affine transformations
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),

    # Convert to tensor
    transforms.ToTensor(),

    # Cutout augmentation
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),

    # Normalize (Note: These values are standard for ImageNet. Adjust if using a different dataset)
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # This may be optional if your images are already this size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard normalization for pretrained models on ImageNet
])


In [17]:
def set_seed(seed_value=42):
    """Set seed for reproducibility."""
    torch.manual_seed(seed_value)  # Set the seed for torch
    torch.cuda.manual_seed(seed_value)  # If you're using GPU
    torch.cuda.manual_seed_all(seed_value)  # If using multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed_value)

In [20]:
import os
current_path = os.getcwd()
print(current_path)
path = current_path + '/drive/MyDrive/Mushrooms/mushroom_data_new/'
print(path)

/content
/content/drive/MyDrive/Mushrooms/mushroom_data_new/


In [30]:
set_seed(42)

# 1. Load all data using ImageFolder
full_dataset = ImageFolder(root=path, transform=train_transform)

# 2. Calculate lengths for train, validation, and test splits
total_size = len(full_dataset)
train_size = int(0.7 * total_size)  # 70% for training
val_size = int(0.2 * total_size)   # 20% for validation
test_size = total_size - train_size - val_size  # 10% for testing

# 3. Use random_split to split the datasets
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Optional: You might want to apply different transformations to validation and test sets (e.g., no augmentations).
# To do this, create a function that modifies the transformations for a given subset of the dataset:
def set_transform(dataset_subset, transform):
    dataset_subset.dataset.transform = transform
    return dataset_subset

train_dataset = set_transform(train_dataset,train_transform)
val_dataset = set_transform(val_dataset, val_transform)  # If you have a separate val_transform without augmentations
test_dataset = set_transform(test_dataset, val_transform)  # Use the same as validation for simplicity


split_path =  '/content/drive/MyDrive/Mushrooms/vit_split_data'


# Saving transformations
with open(split_path+'/train_dataset.pkl', 'wb') as f:
    pickle.dump(train_dataset, f)
with open(split_path+'/val_dataset.pkl', 'wb') as f:
    pickle.dump(val_dataset, f)
with open(split_path+'/test_dataset.pkl', 'wb') as f:
    pickle.dump(test_dataset, f)


# 4. Create DataLoaders for each set
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

1 PRETRAIN model 가져오기

- vit_small_patch16_224: A smaller version of the ViT.
- vit_base_patch16_224: The standard-sized ViT.
- vit_large_patch16_224: A larger version of ViT.
- vit_huge_patch16_224: The biggest commonly available ViT.

중에 선택 가능

In [31]:
model_name = "vit_large_patch16_224"

model = timm.create_model(model_name, pretrained=True)

# Adjust the head of the model for your specific number of classes (e.g., number of mushroom types)
num_classes = len(full_dataset.classes)
model.head = torch.nn.Linear(in_features=model.head.in_features, out_features=num_classes)

print(f'model : {model_name}')
print(f'class num : {num_classes}')

model : vit_large_patch16_224
class num : 215


2 이미 학습된 버섯 모델 파라미터 가져오기

In [ ]:
# Assuming 'model' is the instance of your model
# model.load_state_dict(torch.load('./model_save/mushroom_vit_large_patch16_224.pth'))

In [32]:
# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define a loss function and an optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

In [33]:
# Assuming you have set the device as 'cuda' if available, otherwise 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_epochs = 3
print_every = 10  # Adjust this value to control how often you want to print updates

# Send the model to the device
model = model.to(device)

# Training Loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode

    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training loopW
    for i, (inputs, labels) in tqdm(enumerate(trainloader), total=len(trainloader)):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        if i % print_every == (print_every - 1):  # Print every 'print_every' batches
            interval_accuracy = 100 * correct_train / (print_every * trainloader.batch_size)
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / print_every:.5f}, accuracy: {interval_accuracy:.2f}%")
            running_loss = 0.0
            correct_train = 0  # Reset for the next set of batches


    # train_accuracy = 100 * correct_train / total_train
    # print(f"Epoch {epoch + 1}/{num_epochs}, Training accuracy: {train_accuracy:.2f}%")

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    correct_val = 0
    total_val = 0
    val_loss = 0.0
    with torch.no_grad():  # No gradient needed for validation
        for images, labels in valloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            val_loss += loss.item()

    val_accuracy = 100 * correct_val / total_val
    average_val_loss = val_loss / len(valloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {average_val_loss:.5f}, Validation accuracy: {val_accuracy:.2f}%")

print("Finished Training!")

  0%|          | 0/279 [00:00<?, ?it/s]

OutOfMemoryError: ignored

In [ ]:
# Saving the model's state_dict
torch.save(model.state_dict(), './model_save/mushroom_vit_large_patch16_224_2.pth')
